In [1]:
import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_columns = False

In [4]:
import itertools

In [5]:
df = pd.read_csv('./Datos/Base_Rate_Share.csv', sep=';')

In [6]:
df.rename(columns={
    'RANGO_REC':'rangoRec', 
    'RANGO_ANT':'rangoAnt', 
    'SEGMENTACION':'segmentacion'  
}, inplace=True)

In [7]:
variables_agrupacion = ['SEGMENTACION']

In [8]:
def rate_share_grupos(df, variables_agrupacion):

    df_agrupado = df.groupby(variables_agrupacion).agg({
        'OPENING': ['sum'],
        'ENDING': ['sum'],
        'CHURN': ['sum']
    }).reset_index()

    variables_agrupacion.extend(['OPENING', 'ENDING', 'CHURN'])
    df_agrupado.columns = variables_agrupacion
    df_agrupado['BASE_MEDIA'] = (df_agrupado['OPENING'] + df_agrupado['ENDING'])/2
    df_agrupado['RATE'] = round((df_agrupado['CHURN'] / df_agrupado['BASE_MEDIA']) * 100, 2)
    df_agrupado['SHARE'] = df_agrupado['CHURN'].apply(lambda x: round(x/df_agrupado['CHURN'].sum()*100, 2))
    return df_agrupado

In [29]:
elementos = ['segmentacion']

In [30]:
resultados = [list(i) for i in list(itertools.permutations(elementos, len(elementos)))]

In [33]:
combinaciones = []

for i in range(3):
    for j in range(len(elementos)):
        resultados = [list(j) for j in list(itertools.combinations(elementos, j+1))]
        for k in resultados:
            formateo = ' '.join("_").join(k)
            combinaciones.append(formateo)
            
            df_segmentado = rate_share_grupos(df, k)

            df_segmentado.to_csv(f'./Datos/Sintetizados/{formateo}.csv', index=False)
